# Homework 3
### Caroline Nelson

### Question 1: Read in the data
I know there was a simpler way to read the data, but my initial approach was to create lists for each column, and combined the lists into one data frame.

In [4]:
import pandas as pd
from pandas import Series,DataFrame
df=pd.read_csv('gold.txt',sep='\t')
df.columns=['URL','Category']
df[:10]
# line=fp.readlines()
# category_list=[]
# url_list=[]

# for l in line:
#     url=l.split('\t')
#     category=url[1].split('\n')
#     url_list.append(str(url[0]))
#     category_list+=(category[0])
# df_gold=DataFrame([url_list,category_list])
# df_gold=df_gold.T
# df_gold.columns=['URL','Category']
# df_gold[:10]



,URL,Category
0,http://18games.net,X
1,http://1pixelout.net,G
2,http://1porno.com/004/teens/index.html,X
3,http://1stpussy.com/005/16/index.html,X
4,http://3x-pix.com,X
5,http://4d.com,G
6,http://4neurons.com,G
7,http://4-panama-city-beach-condo.com/index.htm,G
8,http://4whw.8m.net/,X
9,http://6lyrics.com,G


In [4]:
fp2=open('labels.txt','rU')
line2=fp2.readlines()
labels_turk=[]
labels_cat=[]
labels_url=[]

for l in line2:
    remove_n=l.rstrip()
    turk=remove_n.split('\t')
    labels_turk.append(turk[0])
    labels_cat.append(turk[2])
    labels_url.append(turk[1])

df_label=DataFrame([labels_turk,labels_url,labels_cat])
df_label=df_label.T
df_label.columns=['turk','URL','Category']
df_label[:10]

,turk,URL,Category
0,A1OT3A29R9N1DG,http://000.cc,P
1,A1PXXEOGQ76RNJ,http://000.cc,G
2,A1PXXEOGQ76RNJ,http://000.cc,G
3,A21US576U8SCO4,http://000.cc,G
4,A2LGX47NN7C5D3,http://000.cc,G
5,A2OVKTB7VNY8EW,http://000.cc,G
6,A31Z3E5SLATLML,http://000.cc,P
7,AAVCPZ8WTCFVK,http://000.cc,G
8,AHY98D5P05XIV,http://000.cc,G
9,APTFS1S88K5VC,http://000.cc,G


### Question 2: Divide labels into two data frames

In [5]:
mask=df_label['URL'].isin(df_gold['URL'])
labels_on_gold=df_label[mask]
labels_on_gold[:10]

,turk,URL,Category
245,A1253FXHCZ9CWM,http://0800-horoscope.com,G
246,A153PKAL7OAY36,http://0800-horoscope.com,G
247,A1FV9SAPL5C6KY,http://0800-horoscope.com,G
248,A1JTOT0DWM6QGL,http://0800-horoscope.com,G
249,A1PXXEOGQ76RNJ,http://0800-horoscope.com,G
250,A21US576U8SCO4,http://0800-horoscope.com,G
251,A22C0PJUBFJTI0,http://0800-horoscope.com,G
252,A270HOAHEVIFW0,http://0800-horoscope.com,G
253,A2EG8FHODTEV63,http://0800-horoscope.com,R
254,A2LM400F4F0V0M,http://0800-horoscope.com,G


In [6]:
labels_unknown=df_label[-mask]
labels_unknown[:10]

,turk,URL,Category
0,A1OT3A29R9N1DG,http://000.cc,P
1,A1PXXEOGQ76RNJ,http://000.cc,G
2,A1PXXEOGQ76RNJ,http://000.cc,G
3,A21US576U8SCO4,http://000.cc,G
4,A2LGX47NN7C5D3,http://000.cc,G
5,A2OVKTB7VNY8EW,http://000.cc,G
6,A31Z3E5SLATLML,http://000.cc,P
7,AAVCPZ8WTCFVK,http://000.cc,G
8,AHY98D5P05XIV,http://000.cc,G
9,APTFS1S88K5VC,http://000.cc,G


### Question 3: Compute accuracies of turks

In [37]:
merged=labels_on_gold.merge(df_gold,on='URL')
#First, we merge on 'url' to see the different ratings of each 'url'
mask=(merged['Category_x']==merged['Category_y'])
new_merged=(merged[mask])

num_correct=new_merged.groupby(['turk'])['Category_x'].agg(['count'])
#this data frame counts the number correct for each turk
total=labels_on_gold.groupby(['turk'])['Category'].agg(['count'])
#this data frame counts the number of ratings for each turk
rater_goodness=num_correct.merge(total,left_index=True,right_index=True)
#merges num_correct with total, indexed on turk
rater_goodness['Total']=total
rater_goodness['Average Correctness']=rater_goodness['count_x']/rater_goodness['Total']
del rater_goodness['count_y']
del rater_goodness['count_x']
rater_goodness=rater_goodness.reset_index()
rater_goodness[:10]

,turk,Total,Average Correctness
0,A112DVP1KG4QZU,1,1.000000
1,A1253FXHCZ9CWM,29,0.517241
2,A12CY1Q7XKJJDE,1,1.000000
3,A12RE8G66WTO8B,20,0.750000
4,A12Y1GTGIQDGRA,3,0.333333
5,A13CEW9JGDWGX1,1,1.000000
6,A13OE9GBRJ0S2U,4,0.750000
7,A14IQ4GLNWNPOJ,1,1.000000
8,A153PKAL7OAY36,148,0.722973
9,A1554ZM0CLKSG5,1,1.000000


### Question 4: Odds ratios
Taking the ratio of number correct vs. total ratings computed above, I computed the odds ratio (p/1.001-p) for each turk.

In [38]:
rater_goodness['Odds']=(rater_goodness['Average Correctness']/(1.001-rater_goodness['Average Correctness']))
rater_goodness[:10]

,turk,Total,Average Correctness,Odds
0,A112DVP1KG4QZU,1,1.000000,1000.000000
1,A1253FXHCZ9CWM,29,0.517241,1.069214
2,A12CY1Q7XKJJDE,1,1.000000,1000.000000
3,A12RE8G66WTO8B,20,0.750000,2.988048
4,A12Y1GTGIQDGRA,3,0.333333,0.499251
5,A13CEW9JGDWGX1,1,1.000000,1000.000000
6,A13OE9GBRJ0S2U,4,0.750000,2.988048
7,A14IQ4GLNWNPOJ,1,1.000000,1000.000000
8,A153PKAL7OAY36,148,0.722973,2.600369
9,A1554ZM0CLKSG5,1,1.000000,1000.000000


### Question 5: Most Accurate Turks

In [39]:
import matplotlib
mask=(rater_goodness['Total']>=20)
rater_goodness2=rater_goodness[mask]
#This mask reveals the turks with more than 20 ratings
rater_goodness2=rater_goodness2.set_index('turk')
#Since I reset the index in a previous question, I wanted the turk name instead of the corresponding numerical index
rank=rater_goodness2['Average Correctness'].sort_values(ascending=False)
print rank[:10]

turk
A2U0R4X38GUKZE    0.950000
A22C0PJUBFJTI0    0.916667
A23YQUBXZPKILZ    0.875000
ATVALOQVDCMZW     0.854369
A1HIXWH4OXT8S4    0.825000
A3220HG1O83HQ4    0.818182
A20PWAB7G3HDHU    0.800000
A32W20KGQXS0LL    0.800000
AJSJVK40F5HM6     0.785714
A31OCN4MNHUQ6W    0.777174
Name: Average Correctness, dtype: float64


### Question 6: Ratings vs. Accuracy
In order to compare the number of ratings with their accuracy, to see if there is a positive correlation, I compared the range of ratios (number correct vs. total) of turks who have rated more than 20 to those who rated more than 50, to see if the ranges were similar or different.  If they are drastically different, it would mean the more ratings given, the more accurate they become, and the smaller ratios of average correctness would disappear.

In [32]:
mask2=rater_goodness['Total']>=50
rater_goodness3=rater_goodness[mask2]
rater_goodness3=rater_goodness3.set_index('turk')
rank2=rater_goodness3['Average Correctness'].sort_values(ascending=False)
print rank2[:10]
print rank2.max()-rank2.min()
print rank.max()-rank.min()

turk
ATVALOQVDCMZW     0.854369
A31OCN4MNHUQ6W    0.777174
A3MA9V9UPGCOK7    0.774648
A35GER5TWMH9VP    0.750000
APYV9Y0DFZFVP     0.746479
A153PKAL7OAY36    0.722973
A2AJO7WHWIKEHR    0.705882
A28ZHJ9ASQ49IY    0.704918
A2ITXTZLLTI4GO    0.701493
A21A79J5ZWD42K    0.693548
Name: ratio, dtype: float64
0.481487576107
0.847959183673


The range of the ratios for the turks with over 20 ratings (rank) is 0.848, while the range of the ratios for the turks with over 50 ratings (rank2) is 0.481.  The more ratings, the higher the accuracy ratios, the more specialized the turks are.

### Question 7: Overall Predicted Odds
In this question, I first masked rater_goodness based on the total number of ratings being greater than the 75th percentile (11, in this case).  Then I merged the new rater_goodness with labels_unknown to line up odds ratios with turk, and finally, the below pivot table gives the product of odds ratios for each turk given each URL, category pairing.  

In [19]:
import numpy as np
mask=rater_goodness['Total']>=rater_goodness['Total'].quantile(q=0.75)
top75=rater_goodness[mask]
#top75 is the 75th percentile of total number of ratings
merg=labels_unknown.merge(top75,on='turk')
#I merge labels_unknown with the top75 on turk to get the odds column from rater_goodness 
#translated to the turks in labels_unknown
overall_odds=merg.pivot_table(index='URL',columns='Category',values='Odds', aggfunc=np.prod).fillna(0)
overall_odds[:10]

Category,G,P,R,X
URL,,,,
http://0-101.net,2.155963,0.000000,0.00000,0.000000
http://000.cc,1.460583,0.000000,0.00000,0.000000
http://0000.jp,10.373583,0.000000,0.00000,0.000000
http://000relationships.com,2.912430,1.851852,0.00000,0.000000
http://000vitamins.com,2.710047,0.000000,0.00000,0.000000
http://000webhost.com,11.159502,0.000000,0.00000,0.000000
http://003my.com,4.912040,0.000000,0.00000,0.000000
http://007absolutehardcore.com/early_cumshots/index.html,0.000000,3.346522,12.29045,293.252871
http://007swz.cn,1.393883,0.000000,0.00000,0.000000


### Question 8: Predicted Categories

For this question, we need to index overall_odds on URL, and find the top category (category with highest odds) for each URL, and top odds (odds for that top category).

In [12]:
result_75=DataFrame(overall_odds.ix[:,0].index)
#create a data frame from overall_odds, indexed on URL
result_75['G']=overall_odds.ix[:,0].values
result_75['P']=overall_odds.ix[:,1].values
result_75['R']=overall_odds.ix[:,2].values
result_75['X']=overall_odds.ix[:,3].values
#gets values from each category, URL pairing
result_75['Top Odds']=result_75[['G','P','R','X']].max(axis=1)
#picks largest odds ratio per URL
result_75['Top Category']=result_75[['G','P','R','X']].idxmax(axis=1)
#picks category with largest odds ratio per URL
result_75=result_75[['URL','Top Odds','Top Category']]
result_75[:10]


,URL,Top Odds,Top Category
0,http://0-101.net,2.155963,G
1,http://000.cc,1.460583,G
2,http://0000.jp,10.373583,G
3,http://000relationships.com,2.912430,G
4,http://000vitamins.com,2.710047,G
5,http://000webhost.com,11.159502,G
6,http://003my.com,4.912040,G
7,http://007absolutehardcore.com/early_cumshots/...,293.252871,X
8,http://007swz.cn,1.393883,G
9,http://01768.com,1.393883,G


### Question 9: Predicted categories using more turks

In this question, I repeated questions 7 and 8 for the 25th percentile.  Then compare the 75th and 25th percentiles using a crosstab table to count the number of URLs predicted per category at each percentile. 

In [35]:
import numpy as np
mask=rater_goodness['Total']>=rater_goodness['Total'].quantile(q=0.25)
top25=rater_goodness[mask]

merg1=labels_unknown.merge(top25,on='turk')

overall_odds1=merg1.pivot_table(index='URL',columns='Category',values='Odds', aggfunc=np.prod).fillna(0)

result_25=DataFrame(overall_odds1.ix[:,0].index)
result_25['G']=overall_odds1.ix[:,0].values
result_25['P']=overall_odds1.ix[:,1].values
result_25['R']=overall_odds1.ix[:,2].values
result_25['X']=overall_odds1.ix[:,3].values

result_25['Top Odds']=result_25[['G','P','R','X']].max(axis=1)
result_25['Top Category']=result_25[['G','P','R','X']].idxmax(axis=1)
result_25=result_25[['URL','Top Odds','Top Category']]
result_25[:10]


,URL,Top Odds,Top Category
0,http://0-101.net,2.155963e+00,G
1,http://000.cc,2.181050e+03,G
2,http://0000.jp,2.877556e+07,G
3,http://000relationships.com,2.739423e+07,G
4,http://000vitamins.com,5.014149e+00,G
5,http://000webhost.com,1.113723e+07,G
6,http://003my.com,4.912040e+00,G
7,http://007absolutehardcore.com/early_cumshots/...,5.847515e+02,X
8,http://007swz.cn,1.391101e+00,G
9,http://01768.com,1.393883e+00,G


In the table above, the top odds are very large, possibly due to many odds ratios of 1000 where the turk only rated one url, and rated it correctly.  The 25th percentile dealt with those who had only ranked one url, after all.

In [36]:
result_25=result_25[['URL','Top Category']]
result_75=result_75[['URL','Top Category']]
results=pd.merge(result_25,result_75,on='URL')

table=pd.crosstab(index=results['Top Category_x'],columns=results['Top Category_y'])
table

Top Category_y,G,P,R,X
Top Category_x,,,,
G,8252,237,31,24
P,514,352,40,8
R,162,59,126,23
X,224,25,25,470


In this table, Top Category_x is a prediction from the 25th percentile, and Top Category_y is a prediction from the 75th percentile.  It looks like the biggest discrepancies were between G and P ratings.  It seems that most turks agreed on G and X, but there were more discrepancies among ratings somewhat close together, as we would expect.